In [71]:
from pyspark.sql import SparkSession, dataframe
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql import HiveContext

from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import when

spark = SparkSession.builder.master("local[*]")\
    .enableHiveSupport()\
    .getOrCreate()

In [104]:
#LENDO DADOS SPARK
dataframe_alunos_csv =  spark.read.option("header","true").option("inferSchema" , "true")\
        .csv("/user/aluno/caiua/dados_aluno/alunos.csv")

dataframe_alunos_csv.printSchema()

#dataframe_alunos.show()



#df.write.mode("overwrite").saveAsTable("curso_bigdata_indra.aluno_curso")

root
 |-- id_discente: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- ano_ingresso: integer (nullable = true)
 |-- periodo_ingresso: integer (nullable = true)
 |-- nivel: string (nullable = true)
 |-- id_forma_ingresso: integer (nullable = true)
 |-- id_curso: integer (nullable = true)



In [93]:
# PEGANDO DADOS DO HIVE
dataframe_alunos = spark.sql("SELECT * FROM desafio_hive.tbl_alunos")

# TRATAMENTO DE DADOS COM DATAFRAME
dataframe_alunos = dataframe_alunos.withColumn('nome', regexp_replace('nome','\"', ''))

dataframe_alunos.withColumn("nivel", when(dataframe_alunos.nivel == "G","Graduação")
      .when(dataframe_alunos.nivel == "M","MATERNAL INTEGRADO")
      .otherwise('Não Informado')).show()

+-----------+--------------------+------------+----------------+------------------+-----------------+--------+----------+
|id_discente|                nome|ano_ingresso|periodo_ingresso|             nivel|id_forma_ingresso|id_curso|   dt_foto|
+-----------+--------------------+------------+----------------+------------------+-----------------+--------+----------+
|      14627|MATHEUS OLIVEIRA ...|        2015|               1|         Graduação|            34110|  181097|2022-05-28|
|      15754|MATHEUS OLIVEIRA ...|        2016|               1|         Graduação|            62151| 1304109|2022-05-28|
|      15940| MATHEUS PAZ ANGLADA|        2017|                |MATERNAL INTEGRADO|            37350|    5143|2022-05-28|
|      22332|MATHEUS PELISSON ...|        2018|               0|     Não Informado|          2081113| 2399346|2022-05-28|
|      11692|MATHEUS PEREIRA C...|        2016|                |MATERNAL INTEGRADO|            37350|    3402|2022-05-28|
|      23381|MATHEUS PIN

In [132]:
dataframe_cursos = spark.sql("SELECT * FROM desafio_hive.tbl_cursos")

In [94]:
# TRATAMENTO DE DADOS USANDO SPARKSQL
spark.sql('''SELECT
              id_discente,
              trim(replace(nome, '\"', '')) as nome,
              CASE 
                  WHEN nivel = "G" THEN 'GRADUAÇÃO'
                  WHEN nivel = "M" THEN 'MATERNAL INTEGRADO'
                  WHEN nivel = "N" THEN 'MATERNAL INTEGRADO II'
                  WHEN nivel = "T" THEN 'SUBSEQUENTE'
                  WHEN nivel = "L" THEN 'POS - LATU SENSO'
                  WHEN nivel = "M" THEN 'POS - STRICTO SENSO'
                  ELSE 'NÃO INFORMADO' 
              END  AS nivel,
              ano_ingresso,
              CASE 
                  WHEN periodo_ingresso = 0 THEN 'PRIMEIRO PERIODO'
                  WHEN periodo_ingresso = 1 THEN 'SEGUNDO PERIODO'
                  WHEN periodo_ingresso = 2 THEN 'TERCEITO PERIODO'
                  WHEN periodo_ingresso = 3 THEN 'QUARTO PERIODO'
                  ELSE 'NÃO INFORMADO' 
              END as periodo_ingresso,
              id_forma_ingresso,
              id_curso,
              dt_foto
              FROM desafio_hive.tbl_alunos'''
         ).show()

+-----------+--------------------+--------------------+------------+----------------+-----------------+--------+----------+
|id_discente|                nome|               nivel|ano_ingresso|periodo_ingresso|id_forma_ingresso|id_curso|   dt_foto|
+-----------+--------------------+--------------------+------------+----------------+-----------------+--------+----------+
|      14627|MATHEUS OLIVEIRA ...|           GRADUAÇÃO|        2015| SEGUNDO PERIODO|            34110|  181097|2022-05-28|
|      15754|MATHEUS OLIVEIRA ...|           GRADUAÇÃO|        2016| SEGUNDO PERIODO|            62151| 1304109|2022-05-28|
|      15940| MATHEUS PAZ ANGLADA|  MATERNAL INTEGRADO|        2017|   NÃO INFORMADO|            37350|    5143|2022-05-28|
|      22332|MATHEUS PELISSON ...|MATERNAL INTEGRAD...|        2018|PRIMEIRO PERIODO|          2081113| 2399346|2022-05-28|
|      11692|MATHEUS PEREIRA C...|  MATERNAL INTEGRADO|        2016|   NÃO INFORMADO|            37350|    3402|2022-05-28|
|      2

In [9]:
spark.sql("SELECT * FROM desafio_hive.tbl_unidades").show()

+----------+--------------------+--------+-------------------+------------+-----+----------+
|id_unidade|                nome|   sigla|unidade_responsavel|id_municipio|ativo|   dt_foto|
+----------+--------------------+--------+-------------------+------------+-----+----------+
|       118|"ASSESSORIA DE PR...|     APP|                116|        9240|     |2022-05-27|
|       105|"ASSESSORIA DE RE...|     ARI|                110|        9240|     |2022-05-27|
|       679|"ASSESSORIA JURID...|     ASJ|                 51|        9240|     |2022-05-27|
|       177|    "ASSESSORIAS AL"|   ASSAL|                174|        8550|    1|2022-05-27|
|       184|    "ASSESSORIAS AL"|  ASSEAL|                174|        8550|     |2022-05-27|
|       756|    "ASSESSORIAS FW"|   ASSFW|                681|        8883|    1|2022-05-27|
|       765|    "ASSESSORIAS JA"|   ASSJA|                372|        8960|    1|2022-05-27|
|       416|    "ASSESSORIAS JC"|   ASSJC|                412|        

In [136]:
dataframe_alunos.filter("ano_ingresso > 2016").show()

+-----------+--------------------+------------+----------------+-----+-----------------+--------+
|id_discente|                nome|ano_ingresso|periodo_ingresso|nivel|id_forma_ingresso|id_curso|
+-----------+--------------------+------------+----------------+-----+-----------------+--------+
|      18957|ABELARDO DA SILVA...|        2017|               1|    G|            62151|   76995|
|      17977|ABIGAIL ANTUNES S...|        2017|               1|    T|          2081111|  759711|
|      16613|ABIGAIL FERNANDA ...|        2017|            null|    M|            37350|    1222|
|      17398|ABIGAIL JOSIANE R...|        2017|            null|    M|            37350|    5041|
|      26880|ABIMAEL CHRISTOPF...|        2019|               1|    T|          2081115| 1913420|
|      28508|   ABNER NUNES PERES|        2019|               1|    G|            37350|  181097|
|      18693|ACSA PEREIRA RODR...|        2017|               1|    G|            62151|   77498|
|      28071|ACSA RO